Hypothesis: University towns have their mean housing prices less effected by recessions. 

Run a t-test to compare the ratio of the mean price of houses in university towns the quarter before the recession starts compared to the recession bottom. (price_ratio=quarter_before_recession/recession_bottom)


Available Data files:

1-From the Zillow research data site there is housing data for the United States. In particular the datafile for all homes at a city level, 'City_Zhvi_AllHomes.csv', has median home sale prices at a fine grained level.

2-From the Wikipedia page on college towns,'university_towns.txt' is a list of university towns in the US.A university town is a city which has a high percentage of university students compared to the total population of the city.

3-From Bureau of Economic Analysis, US Department of Commerce, the GDP over time of the US in current dollars (use the chained value in 2009 dollars), in quarterly intervals, in the file 'gdplev.xls'. Only look at GDP data from the first quarter of 2000 onward.


Note: 

1-Recession starts with two consecutive quarters of GDP decline, and ends with two consecutive quarters of GDP growth.

2-A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.

3-A recession bottom is the quarter within a recession which had the lowest GDP.

4-t-test is is used to determine whether the mean of homes'price in university town differs significantly from the mean of homes' price in non-university town.

In [ ]:
import pandas as pd
import numpy as np
from scipy.stats import ttest_ind

Question 1.
Read the 'university_towns.txt'and do the following cleaning:

    1. For "State", removing characters from "[" to the end.
    2. For "RegionName", when applicable, removing every character from " (" to the end.
    3. Depending on how you read the data, you may need to remove newline character '\n'. '''

to Return a DataFrame that contains two columns=["State", "RegionName"] 

In [68]:
def get_list_of_university_towns():
    uni_town=pd.read_csv('university_towns.txt',sep='\n', header=None, names=['RegionName'])
    uni_town['State']=np.where (uni_town['RegionName'].str.contains('edit'), uni_town['RegionName'], np.NaN)
    uni_town['State']=uni_town['State'].fillna(method='ffill')
    uni_town=uni_town[['State','RegionName']]
    for col in uni_town:
        uni_town[col]=uni_town[col].str.split('(', expand=True)[0].str.split('[',expand=True)[0].str.rstrip()
    
    uni_town=uni_town[uni_town['State']!=uni_town['RegionName']]
    return uni_town

get_list_of_university_towns()   

,State,RegionName
1,Alabama,Auburn
2,Alabama,Florence
3,Alabama,Jacksonville
4,Alabama,Livingston
5,Alabama,Montevallo
...,...,...
561,Wisconsin,River Falls
562,Wisconsin,Stevens Point
563,Wisconsin,Waukesha
564,Wisconsin,Whitewater


Question 2.

Load 'gdplev.xls'; Returns the year and quarter of the recession start time as a string value in a format such as 2005q3.only look at GDP data from the first quarter of 2000 onward.

Definitions:

A recession is defined as starting with two consecutive quarters of GDP decline, and ending with two consecutive quarters of GDP growth.
A recession bottom is the quarter within a recession which had the lowest GDP.

In [69]:
def get_recession_start():
    gdp=pd.read_excel ('gdplev.xls', skiprows=5, usecols=[4,6])
    gdp=gdp.dropna()
    gdp.columns=['yrQrt','GDP']
    gdp['consequtive_diff']=gdp['GDP'].diff()
    m=int(gdp.index[gdp['yrQrt'].str.contains('2000q1')][0])
    GDP=gdp.loc[m:]
    Recc=GDP.where(GDP['consequtive_diff']<0).dropna()
    Recc['Recc.index']=Recc.index
    Recc['diff_index']=Recc['Recc.index'].diff()
    return Recc.loc[Recc['diff_index'].idxmin()-1 , 'yrQrt']

get_recession_start()

'2008q3'

Question3.

Returns the year and quarter of the recession end time as a string value in a format such as 2005q3.
Note: Recession starts with two consecutive quarters of GDP decline, and ends with two consecutive quarters of GDP growth.

In [70]:
def get_recession_end():
    gdp=pd.read_excel ('gdplev.xls', skiprows=5, usecols=[4,6])
    gdp=gdp.dropna()
    gdp.columns=['yrQrt','GDP']
    gdp['consequtive_diff']=gdp['GDP'].diff()
    m=int(gdp.index[gdp['yrQrt'].str.contains('2000q1')][0])
    GDP=gdp.loc[m:]
    growth=GDP.where(GDP['consequtive_diff']>0).dropna()
    growth['Recc.index']=growth.index
    growth['diff_index']=growth['Recc.index'].diff()
    return growth.loc[growth['diff_index'].idxmax()+1 , 'yrQrt']

get_recession_end()

'2009q4'

Question 4.

Returns the year and quarter of the recession bottom time as a string value in a format such as 2005q3.

In [71]:
def get_recession_bottom():
    gdp=pd.read_excel ('gdplev.xls', skiprows=5, usecols=[4,6])
    gdp=gdp.dropna()
    gdp.columns=['yrQrt','GDP']
    m=int(gdp.index[gdp['yrQrt'].str.contains('2000q1')][0])
    GDP=gdp.loc[m:]


    a=int(GDP.where(GDP['yrQrt']==get_recession_start()).dropna().index[0])
    b=int(GDP.where(GDP['yrQrt']==get_recession_end()).dropna().index[0])
    Bottom=GDP.loc[a:b,]
    bottom=Bottom['GDP'].min()
    bottom_idx=Bottom.where(Bottom['GDP']==bottom).dropna().index[0]
    
    return Bottom.loc[bottom_idx,'yrQrt']
get_recession_bottom()

'2009q2'

Question 5.

Load 'City_Zhvi_AllHomes.csv'from the Zillow research data site. It is a the datafile for all homes at a city level, has median home sale prices at a fine grained level. Converts the housing data to quarters and returns it as mean values in a dataframe. This dataframe should be a dataframe with columns for 2000q1 through 2016q3, and should have a multi-index in the shape of ["State","RegionName"].
The resulting dataframe should have 67 columns, and 10,730 rows.


In [72]:
states = {'OH': 'Ohio', 'KY': 'Kentucky', 'AS': 'American Samoa', 'NV': 'Nevada', 
          'WY': 'Wyoming', 'NA': 'National', 'AL': 'Alabama', 'MD': 'Maryland', 'AK': 'Alaska',
          'UT': 'Utah', 'OR': 'Oregon6tr', 'MT': 'Montana', 'IL': 'Illinois', 'TN': 'Tennessee', 
          'DC': 'District of Columbia', 'VT': 'Vermont', 'ID': 'Idaho', 'AR': 'Arkansas', 'ME': 'Maine', 
          'WA': 'Washington', 'HI': 'Hawaii', 'WI': 'Wisconsin', 'MI': 'Michigan', 'IN': 'Indiana', 
          'NJ': 'New Jersey', 'AZ': 'Arizona', 'GU': 'Guam', 'MS': 'Mississippi', 'PR': 'Puerto Rico', 
          'NC': 'North Carolina', 'TX': 'Texas', 'SD': 'South Dakota', 'MP': 'Northern Mariana Islands',
          'IA': 'Iowa', 'MO': 'Missouri', 'CT': 'Connecticut', 'WV': 'West Virginia', 'SC': 'South Carolina', 
          'LA': 'Louisiana', 'KS': 'Kansas', 'NY': 'New York', 'NE': 'Nebraska', 'OK': 'Oklahoma', 'FL': 'Florida', 
          'CA': 'California', 'CO': 'Colorado', 'PA': 'Pennsylvania', 'DE': 'Delaware', 'NM': 'New Mexico', 
          'RI': 'Rhode Island', 'MN': 'Minnesota', 'VI': 'Virgin Islands', 'NH': 'New Hampshire', 
          'MA': 'Massachusetts', 'GA': 'Georgia', 'ND': 'North Dakota', 'VA': 'Virginia'}

In [73]:
def convert_housing_data_to_quarters():
    HomePrice=pd.read_csv('City_Zhvi_AllHomes.csv',header=0)                        
    HomePrice=HomePrice.drop(HomePrice.columns[HomePrice.columns.get_loc('1996-04'):HomePrice.columns.get_loc('2000-01')], 
                         axis=1)
    HomePrice=HomePrice.drop (HomePrice.columns[[0,3,4,5]], axis=1)
    HomePrice['State']=HomePrice['State'].replace(to_replace=states)
    HomePrice=HomePrice.set_index(['State','RegionName'])

    def yrqrt(s:str):
        m=s.split('-')  
        yr=m[0]
        qrt=str(int((int(m[1])-1)/3)+1)
        return (yr+'q'+qrt)


    cols=HomePrice.columns
    colname=[]
    for col in cols:
        colname.append(yrqrt(col))
    
    HomePrice.columns=colname

    HomePrice=HomePrice.groupby(HomePrice.columns,axis=1).mean()
    return HomePrice
convert_housing_data_to_quarters()

,,2000q1,2000q2,2000q3,2000q4,2001q1,2001q2,2001q3,2001q4,2002q1,2002q2,...,2014q2,2014q3,2014q4,2015q1,2015q2,2015q3,2015q4,2016q1,2016q2,2016q3
State,RegionName,,,,,,,,,,,,,,,,,,,,,
New York,New York,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,515466.666667,522800.000000,528066.666667,532266.666667,540800.000000,557200.000000,572833.333333,582866.666667,591633.333333,587200.0
California,Los Angeles,207066.666667,214466.666667,220966.666667,226166.666667,233000.000000,239100.000000,245066.666667,253033.333333,261966.666667,272700.000000,...,498033.333333,509066.666667,518866.666667,528800.000000,538166.666667,547266.666667,557733.333333,566033.333333,577466.666667,584050.0
Illinois,Chicago,138400.000000,143633.333333,147866.666667,152133.333333,156933.333333,161800.000000,166400.000000,170433.333333,175500.000000,177566.666667,...,192633.333333,195766.666667,201266.666667,201066.666667,206033.333333,208300.000000,207900.000000,206066.666667,208200.000000,212000.0
Pennsylvania,Philadelphia,53000.000000,53633.333333,54133.333333,54700.000000,55333.333333,55533.333333,56266.666667,57533.333333,59133.333333,60733.333333,...,113733.333333,115300.000000,115666.666667,116200.000000,117966.666667,121233.333333,122200.000000,123433.333333,126933.333333,128700.0
Arizona,Phoenix,111833.333333,114366.666667,116000.000000,117400.000000,119600.000000,121566.666667,122700.000000,124300.000000,126533.333333,128366.666667,...,164266.666667,165366.666667,168500.000000,171533.333333,174166.666667,179066.666667,183833.333333,187900.000000,191433.333333,195200.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Wisconsin,Town of Wrightstown,101766.666667,105400.000000,111366.666667,114866.666667,125966.666667,129900.000000,129900.000000,129433.333333,131900.000000,134200.000000,...,144866.666667,146866.666667,149233.333333,148666.666667,149333.333333,149866.666667,149933.333333,149833.333333,151266.666667,155000.0
New York,Urbana,79200.000000,81666.666667,91700.000000,98366.666667,94866.666667,98533.333333,102966.666667,98033.333333,93966.666667,94600.000000,...,132133.333333,137033.333333,140066.666667,141700.000000,137866.666667,136466.666667,136166.666667,138966.666667,144200.000000,143000.0
Wisconsin,New Denmark,114566.666667,119266.666667,126066.666667,131966.666667,143800.000000,146966.666667,148366.666667,149166.666667,153133.333333,156733.333333,...,174566.666667,181166.666667,186166.666667,187600.000000,188666.666667,188433.333333,188933.333333,191066.666667,192833.333333,197600.0


Question 6.

First creates new data showing the decline or growth of housing prices between the recession start and the recession bottom.
Then runs a ttest
comparing the university town values to the non-university towns values, return whether the alternative hypothesis (that the two groups are the same) is true or not as well as the p-value of the confidence. 

Note:
- Recession occures in the quarter BEFORE the recorded quarte that recession starts.
-t-test is is used to determine whether the mean of homes'price in university town differs significantly from the mean of homes' price in non-university town.


In [74]:
def run_ttest():
    df=convert_housing_data_to_quarters()
    start_idx=df.columns.get_loc(get_recession_start())-1
    end_idx=df.columns.get_loc(get_recession_bottom())
    df= np.divide(df.iloc[:,start_idx],df.iloc[:,end_idx]).to_frame().dropna()
    
    town=get_list_of_university_towns().set_index(['State', 'RegionName'])
    
    
    uni_Town=df.merge (town, how= 'inner', right_index=True, left_index=True)
    nonuni_Town=df.drop(uni_Town.index)
    
    p_value=ttest_ind(uni_Town.values,nonuni_Town.values).pvalue

    if p_value<0.01:
        different=True
    else:
        different=False
    

    if uni_Town.mean().values < nonuni_Town.mean().values:
        better='university town'
    else:
        better='non-university town'
    
    
    return (different, p_value[0], better)
run_ttest()

(True, 0.0019686455708254535, 'university town')